In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
# exp_names = ["1698157809_Paper_v3_Real30", "1698169547_Paper_v3_Real30_Laplace"]
exp_names = ["1725451866_paper10CV100tree", "1725746609_paper10CV100treeOOB"]

exp_comp = "_L"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,VA,CT,PPA,Rank,RF_d_L,RF_opt_L,RF_large_L,Platt_L,ISO_L,Beta_L,VA_L,CT_L,PPA_L,Rank_L
Data,,,,,,,,,,,,,,,,,,,,
cm1,0.087722,8.414907e-02,0.086974,0.086919,9.169198e-02,8.645677e-02,0.092338,0.085833,8.511593e-02,0.100365,0.087722,8.414907e-02,0.086974,0.085223,0.085126,8.331389e-02,0.085414,0.085833,8.384972e-02,0.104881
datatrieve,0.082409,8.212649e-02,0.081989,0.089416,9.485366e-02,1.133832e-01,0.104404,0.077444,8.447103e-02,0.086425,0.082409,8.212649e-02,0.081989,0.080482,0.084783,8.242652e-02,0.084055,0.077444,8.309889e-02,0.091607
kc1_class_level_defectiveornot,0.170602,1.695957e-01,0.169659,0.172950,2.116378e-01,1.940572e-01,0.184574,0.172628,1.740477e-01,0.210119,0.170602,1.695957e-01,0.169659,0.170211,0.178321,1.697150e-01,0.175016,0.172628,1.699892e-01,0.219734
kc1,0.103022,1.062869e-01,0.102212,0.108432,1.090643e-01,1.074132e-01,0.108078,0.109514,1.068088e-01,0.114145,0.103022,1.062869e-01,0.102212,0.107295,0.107466,1.062639e-01,0.107117,0.109514,1.062920e-01,0.118342
kc2,0.119612,1.142086e-01,0.119501,0.117958,1.249521e-01,1.189483e-01,0.120627,0.118241,1.176585e-01,0.129250,0.119612,1.142086e-01,0.119501,0.115435,0.116054,1.161339e-01,0.115288,0.118241,1.142152e-01,0.135935
kc3,0.074525,7.337612e-02,0.073146,0.077914,8.708880e-02,7.894980e-02,0.080480,0.074061,7.660898e-02,0.082561,0.074525,7.337612e-02,0.073146,0.074019,0.074731,7.391150e-02,0.073689,0.074061,7.355936e-02,0.086899
pc1,0.049848,5.029936e-02,0.049611,0.052991,5.519468e-02,5.345402e-02,0.054739,0.050945,5.146327e-02,0.060976,0.049848,5.029936e-02,0.049611,0.050831,0.050856,5.122406e-02,0.050384,0.050945,5.031019e-02,0.055856
spect,0.130758,1.210899e-01,0.130126,0.127432,1.382771e-01,1.294192e-01,0.133135,0.121226,1.213519e-01,0.145181,0.130758,1.210899e-01,0.130126,0.120273,0.122023,1.207798e-01,0.121696,0.121226,1.209072e-01,0.153491
spectf,0.125801,1.240775e-01,0.123636,0.129174,1.417815e-01,1.320929e-01,0.129802,0.123573,1.288509e-01,0.140789,0.125801,1.240775e-01,0.123636,0.126119,0.128559,1.244021e-01,0.125905,0.123573,1.250323e-01,0.167081


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):
    print("test_table", test_table)

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [5]:
calib_methods = params["calib_methods"]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        # if method == "RF_opt" and metric == "brier":
        #     print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./comp_stat_test.txt")
# res_table

test_table                                     RF_d    RF_d_L
Data                                              
cm1                             0.894000  0.894000
datatrieve                      0.901538  0.901538
kc1_class_level_defectiveornot  0.744857  0.744857
kc1                             0.860027  0.860027
kc2                             0.834006  0.834006
kc3                             0.897816  0.897816
pc1                             0.938152  0.938152
spect                           0.823390  0.823390
spectf                          0.809801  0.809801
vertebral                       0.836774  0.836774
wilt                            0.982807  0.982807
parkinsons                      0.907842  0.907842
heart                           0.818645  0.818645
wdbc                            0.961347  0.961347
bank                            0.993879  0.993879
ionosphere                      0.932810  0.932810
HRCompetencyScores              0.934667  0.934667
spambase            

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_38045/893604486.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./comp_stat_test.txt")
